In [1]:
import pandas as pd

In [140]:
class feature_engine:
    
    pd = __import__('pandas')
    datetime = __import__('datetime')
    mcal = __import__('pandas_market_calendars')
    np = __import__('numpy')
    
    def __init__(self):
        print('feature_engine V.0.1 \nImported pandas,datetime,pandas_market_calendars,numpy packages')
        
    def calc_price_change(self,tweet_time,financial_data,time_col_name,price_col_name,interval,amount,method='both',
                          logdiff=True):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        tweet_price=financial_data[financial_data[time_col_name]==tweet_time_5min_before][price_col_name]
        if len(tweet_price)==0:
            return float('NaN')
        tweet_price.reset_index(inplace=True,drop=True)

        def calculation(financial_data,time_col_name,new_time,price_col_name,tweet_price,direction,logdiff):
            
            other_price=financial_data[financial_data[time_col_name]==new_time][price_col_name]
            other_price.reset_index(inplace=True,drop=True)
            if len(other_price)==0:
                return float('NaN')
            elif logdiff:
                if direction=='forward':
                    return (self.np.log(other_price.iloc[0]) - self.np.log(tweet_price.iloc[0]))
                elif direction=='backward':
                    return (self.np.log(tweet_price.iloc[0]) - self.np.log(other_price.iloc[0]))
            else:
                if direction=='forward':
                    return ((other_price.iloc[0]/tweet_price.iloc[0])-1)
                elif direction=='backward':
                    return ((tweet_price.iloc[0]/other_price.iloc[0])-1)

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            pct_change_forward=calculation(financial_data,time_col_name,time_forward,price_col_name,tweet_price,'forward',
                                          logdiff)

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            pct_change_backward=calculation(financial_data,time_col_name,time_backward,price_col_name,tweet_price,'backward'
                                           ,logdiff)
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward,
                          f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward},index=[0])
        
    def create_pricechg_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,price_col_name,
                       interval_amount_dict,method='both',logdiff=True):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calc_price_change(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   price_col_name,interval,amount,method,logdiff)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def mean_encoding_tocolumn(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        grouped=features_df.groupby([categorical_column]).mean()
        features_df=features_df.merge(grouped,on=categorical_column)
        return features_df[f'{target_column}_y']
    
    def mean_encoding_todict(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        return features_df.groupby([categorical_column]).mean()
    
    def calculate_sum_volume(self,tweet_time,financial_data,time_col_name,volume_col_name,interval,amount,method='both'):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        def calculation(financial_data,time_col_name,new_time,volume_col_name,tweet_time,direction):
            if direction=='forward':
                other_volume_df=financial_data.loc[(financial_data[time_col_name]<=new_time) & 
                                            (financial_data[time_col_name]>=tweet_time)][volume_col_name]
            elif direction=='backward':
                other_volume_df=financial_data.loc[(financial_data[time_col_name]>=new_time) & 
                                            (financial_data[time_col_name]<=tweet_time)][volume_col_name]
            other_volume=sum(other_volume_df)
            return other_volume

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            vol_sum_forward=calculation(financial_data,time_col_name,time_forward,volume_col_name,tweet_time_5min_before
                                        ,'forward')

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            vol_sum_backward=calculation(financial_data,time_col_name,time_backward,volume_col_name,tweet_time,'backward')
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum_forward,
                          f'{amount}_{interval}_backward_vol_sum':vol_sum_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_vol_sum':vol_sum_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum_forward},index=[0])
        
    def create_volumesum_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,volume_col_name,
                       interval_amount_dict,method='both'):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calculate_sum_volume(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   volume_col_name,interval,amount,method)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def fill_missing_fin_data(self,ticker_col_name,stock_exchg_name,start_date,end_date,frequency,fin_time_colname,
                          financial_data,volume_colname,price_colname):
        ticker_name=financial_data.iloc[0][ticker_col_name]
        stock_exchg = self.mcal.get_calendar(stock_exchg_name, open_time=self.datetime.time(0, 0), 
                                             close_time=self.datetime.time(16, 0))
        daterange = stock_exchg.schedule(start_date, end_date)
        dates=self.pd.DataFrame(self.mcal.date_range(daterange, frequency).tz_convert(None),columns=[fin_time_colname])

#         financial_data[fin_time_colname] = self.pd.to_datetime(financial_data[fin_time_colname], utc = True)

        financial_data=dates.merge(financial_data,on=fin_time_colname,how='left')
        financial_data[ticker_col_name]=ticker_name

        financial_data['SYM_SUFFIX'].fillna(0,inplace=True)

        financial_data[volume_colname].fillna(0,inplace=True)
        financial_data[price_colname].interpolate(inplace=True)
        return financial_data
    
    def token_matrix(self,text_column,financial_topic_words):
        
        def check_for_word(token_vector,word):
            if word in token_vector:
                return 1
            else:
                return 0
            
        new_df=self.pd.DataFrame()
        for topic_word in fin_topic_words:
            new_df[topic_word]=text_column.apply(lambda x:check_for_word(x.lower().split(),topic_word))
            
        return new_df

In [19]:
financial_data=pd.read_csv('04052009-18122019-SPYDIA-minutedata.csv',parse_dates=['DATETIME'])
twitter_data=pd.read_csv('trump_tweets_reducted_after_nlp.csv',index_col=0,parse_dates=['created_at_utc'])

In [20]:
sp500_data=financial_data[financial_data['SYM_ROOT']=='SPY']
dow_data=financial_data[financial_data['SYM_ROOT']=='DIA']

In [141]:
feat_eng = feature_engine()

feature_engine V.0.1 
Imported pandas,datetime,pandas_market_calendars,numpy packages


In [22]:
sp500_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          sp500_data,'SIZE','PRICE')

In [23]:
dow_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          dow_data,'SIZE','PRICE')

In [24]:
tweets_finance=feat_eng.create_pricechg_columns(twitter_data,'created_at_utc',sp500_data,'DATETIME','PRICE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3),('hours',6),('days',1),('days',5),('days',10),('days',30)],method='forward')


0it [00:00, ?it/s]
1it [00:00,  1.46it/s]
2it [00:00,  1.88it/s]
3it [00:01,  2.06it/s]
4it [00:01,  2.55it/s]
5it [00:01,  2.48it/s]
6it [00:02,  2.52it/s]
7it [00:02,  2.57it/s]
8it [00:03,  2.50it/s]
9it [00:03,  2.48it/s]
10it [00:03,  2.98it/s]
11it [00:03,  3.44it/s]
12it [00:04,  2.73it/s]
13it [00:04,  2.67it/s]
14it [00:05,  2.63it/s]
15it [00:05,  2.76it/s]
16it [00:05,  2.89it/s]
17it [00:06,  2.59it/s]
18it [00:06,  3.09it/s]
19it [00:06,  2.98it/s]
20it [00:07,  2.90it/s]
21it [00:07,  2.87it/s]
22it [00:07,  2.85it/s]
23it [00:08,  2.80it/s]
24it [00:08,  3.31it/s]
25it [00:08,  3.79it/s]
26it [00:08,  4.19it/s]
27it [00:09,  3.59it/s]
28it [00:09,  4.05it/s]
29it [00:09,  3.02it/s]
30it [00:10,  3.50it/s]
31it [00:10,  3.19it/s]
32it [00:10,  2.92it/s]
33it [00:10,  3.34it/s]
34it [00:11,  2.92it/s]
35it [00:11,  2.82it/s]
36it [00:12,  2.67it/s]
37it [00:12,  2.61it/s]
38it [00:13,  2.32it/s]
39it [00:13,  2.19it/s]
40it [00:14,  2.16it/s]
41it [00:14,  2.14it/s]
42it 

332it [02:03,  2.25it/s]
333it [02:04,  2.11it/s]
334it [02:04,  2.09it/s]
335it [02:05,  2.12it/s]
336it [02:05,  2.20it/s]
337it [02:06,  1.93it/s]
338it [02:06,  1.98it/s]
339it [02:07,  1.98it/s]
340it [02:07,  2.19it/s]
341it [02:07,  2.73it/s]
342it [02:08,  2.64it/s]
343it [02:08,  2.58it/s]
344it [02:08,  3.02it/s]
345it [02:09,  2.58it/s]
346it [02:09,  2.56it/s]
347it [02:10,  2.53it/s]
348it [02:10,  2.58it/s]
349it [02:11,  2.48it/s]
350it [02:11,  3.02it/s]
351it [02:11,  3.42it/s]
352it [02:11,  2.91it/s]
353it [02:12,  2.75it/s]
354it [02:12,  2.70it/s]
355it [02:13,  2.52it/s]
356it [02:13,  2.34it/s]
357it [02:14,  2.39it/s]
358it [02:14,  2.11it/s]
359it [02:15,  2.08it/s]
360it [02:15,  2.23it/s]
361it [02:15,  2.34it/s]
362it [02:16,  2.86it/s]
363it [02:16,  2.79it/s]
364it [02:16,  2.82it/s]
365it [02:17,  2.81it/s]
366it [02:17,  2.83it/s]
367it [02:17,  2.73it/s]
368it [02:18,  2.71it/s]
369it [02:18,  2.40it/s]
370it [02:19,  2.49it/s]
371it [02:19,  2.57it/s]


659it [04:10,  3.50it/s]
660it [04:10,  3.19it/s]
661it [04:11,  3.08it/s]
662it [04:11,  2.94it/s]
663it [04:11,  3.35it/s]
664it [04:12,  3.79it/s]
665it [04:12,  3.41it/s]
666it [04:12,  3.16it/s]
667it [04:13,  3.05it/s]
668it [04:13,  2.97it/s]
669it [04:13,  2.93it/s]
670it [04:14,  2.88it/s]
671it [04:14,  2.86it/s]
672it [04:14,  2.85it/s]
673it [04:15,  2.78it/s]
674it [04:15,  2.80it/s]
675it [04:15,  2.78it/s]
676it [04:16,  2.82it/s]
677it [04:16,  2.83it/s]
678it [04:17,  2.83it/s]
679it [04:17,  2.80it/s]
680it [04:17,  2.78it/s]
681it [04:18,  2.77it/s]
682it [04:18,  2.72it/s]
683it [04:18,  2.76it/s]
684it [04:19,  2.74it/s]
685it [04:19,  3.15it/s]
686it [04:19,  2.91it/s]
687it [04:20,  2.59it/s]
688it [04:20,  2.31it/s]
689it [04:21,  2.42it/s]
690it [04:21,  2.50it/s]
691it [04:21,  2.57it/s]
692it [04:22,  2.25it/s]
693it [04:23,  2.16it/s]
694it [04:23,  2.21it/s]
695it [04:23,  2.35it/s]
696it [04:24,  2.41it/s]
697it [04:24,  2.22it/s]
698it [04:25,  2.25it/s]


986it [06:16,  2.23it/s]
987it [06:17,  1.99it/s]
988it [06:17,  2.21it/s]
989it [06:18,  2.05it/s]
990it [06:18,  2.07it/s]
991it [06:18,  2.24it/s]
992it [06:19,  2.74it/s]
993it [06:19,  2.75it/s]
994it [06:19,  2.68it/s]
995it [06:20,  2.60it/s]
996it [06:20,  2.65it/s]
997it [06:21,  2.53it/s]
998it [06:21,  2.48it/s]
999it [06:21,  2.50it/s]
1000it [06:22,  2.58it/s]
1001it [06:22,  2.65it/s]
1002it [06:22,  2.67it/s]
1003it [06:23,  2.64it/s]
1004it [06:23,  2.68it/s]
1005it [06:24,  2.70it/s]
1006it [06:24,  2.60it/s]
1007it [06:24,  2.40it/s]
1008it [06:25,  2.00it/s]
1009it [06:26,  2.16it/s]
1010it [06:26,  2.60it/s]
1011it [06:26,  2.20it/s]
1012it [06:27,  2.16it/s]
1013it [06:27,  2.00it/s]
1014it [06:28,  2.03it/s]
1015it [06:28,  2.15it/s]
1016it [06:29,  2.21it/s]
1017it [06:29,  2.66it/s]
1018it [06:29,  2.60it/s]
1019it [06:30,  2.50it/s]
1020it [06:30,  2.46it/s]
1021it [06:31,  2.46it/s]
1022it [06:31,  2.42it/s]
1023it [06:31,  2.28it/s]
1024it [06:32,  2.28it/s]


1301it [08:14,  2.63it/s]
1302it [08:14,  2.58it/s]
1303it [08:15,  2.58it/s]
1304it [08:15,  2.64it/s]
1305it [08:15,  2.67it/s]
1306it [08:16,  2.68it/s]
1307it [08:16,  2.71it/s]
1308it [08:16,  3.24it/s]
1309it [08:16,  3.27it/s]
1310it [08:17,  2.95it/s]
1311it [08:17,  2.67it/s]
1312it [08:18,  2.43it/s]
1313it [08:18,  2.98it/s]
1314it [08:18,  3.51it/s]
1315it [08:18,  4.01it/s]
1316it [08:18,  4.50it/s]
1317it [08:19,  3.82it/s]
1318it [08:19,  4.29it/s]
1319it [08:19,  4.69it/s]
1320it [08:19,  5.02it/s]
1321it [08:19,  5.33it/s]
1322it [08:20,  5.51it/s]
1323it [08:20,  5.62it/s]
1324it [08:20,  4.10it/s]
1325it [08:21,  3.58it/s]
1326it [08:21,  3.31it/s]
1327it [08:21,  3.16it/s]
1328it [08:22,  3.00it/s]
1329it [08:22,  2.95it/s]
1330it [08:22,  2.90it/s]
1331it [08:23,  3.35it/s]
1332it [08:23,  2.75it/s]
1333it [08:23,  2.69it/s]
1334it [08:24,  2.71it/s]
1335it [08:24,  2.67it/s]
1336it [08:25,  2.40it/s]
1337it [08:25,  2.35it/s]
1338it [08:25,  2.78it/s]
1339it [08:2

...

24341it [1:25:04,  4.04it/s]
24342it [1:25:04,  4.02it/s]
24343it [1:25:05,  4.01it/s]
24344it [1:25:05,  4.00it/s]
24345it [1:25:05,  4.76it/s]
24346it [1:25:05,  5.51it/s]
24347it [1:25:05,  6.16it/s]
24348it [1:25:05,  6.77it/s]
24349it [1:25:05,  7.18it/s]
24350it [1:25:06,  7.56it/s]
24351it [1:25:06,  5.92it/s]
24352it [1:25:06,  5.13it/s]
24353it [1:25:06,  4.69it/s]
24354it [1:25:07,  4.45it/s]
24355it [1:25:07,  4.31it/s]
24356it [1:25:07,  4.19it/s]
24357it [1:25:07,  4.13it/s]
24358it [1:25:08,  4.08it/s]
24359it [1:25:08,  4.06it/s]
24360it [1:25:08,  4.02it/s]
24361it [1:25:08,  4.00it/s]
24362it [1:25:09,  3.97it/s]
24363it [1:25:09,  4.75it/s]
24364it [1:25:09,  5.49it/s]
24365it [1:25:09,  6.14it/s]
24366it [1:25:09,  6.66it/s]
24367it [1:25:09,  7.09it/s]
24368it [1:25:09,  7.51it/s]
24369it [1:25:09,  7.76it/s]
24370it [1:25:10,  8.01it/s]
24371it [1:25:10,  8.19it/s]
24372it [1:25:10,  8.29it/s]
24373it [1:25:10,  8.37it/s]
24374it [1:25:10,  6.21it/s]
24375it [1:25:

24623it [1:25:56,  3.78it/s]
24624it [1:25:56,  3.81it/s]
24625it [1:25:56,  3.83it/s]
24626it [1:25:57,  3.70it/s]
24627it [1:25:57,  3.71it/s]
24628it [1:25:57,  3.80it/s]
24629it [1:25:57,  3.84it/s]
24630it [1:25:58,  3.82it/s]
24631it [1:25:58,  3.82it/s]
24632it [1:25:58,  4.49it/s]
24633it [1:25:58,  5.25it/s]
24634it [1:25:58,  5.94it/s]
24635it [1:25:58,  6.54it/s]
24636it [1:25:59,  7.00it/s]
24637it [1:25:59,  7.46it/s]
24638it [1:25:59,  7.72it/s]
24639it [1:25:59,  7.94it/s]
24640it [1:25:59,  8.11it/s]
24641it [1:25:59,  6.15it/s]
24642it [1:25:59,  5.29it/s]
24643it [1:26:00,  4.83it/s]
24644it [1:26:00,  4.51it/s]
24645it [1:26:00,  4.33it/s]
24646it [1:26:00,  4.21it/s]
24647it [1:26:01,  4.15it/s]
24648it [1:26:01,  4.08it/s]
24649it [1:26:01,  4.02it/s]
24650it [1:26:01,  3.99it/s]
24651it [1:26:02,  3.96it/s]
24652it [1:26:02,  3.95it/s]
24653it [1:26:02,  3.94it/s]
24654it [1:26:03,  3.92it/s]
24655it [1:26:03,  3.91it/s]
24656it [1:26:03,  3.92it/s]
24657it [1:26:

24905it [1:26:47,  3.93it/s]
24906it [1:26:48,  3.95it/s]
24907it [1:26:48,  4.73it/s]
24908it [1:26:48,  5.47it/s]
24909it [1:26:48,  6.15it/s]
24910it [1:26:48,  5.29it/s]
24911it [1:26:49,  4.80it/s]
24912it [1:26:49,  4.52it/s]
24913it [1:26:49,  4.33it/s]
24914it [1:26:49,  4.19it/s]
24915it [1:26:50,  4.11it/s]
24916it [1:26:50,  4.04it/s]
24917it [1:26:50,  4.02it/s]
24918it [1:26:50,  4.02it/s]
24919it [1:26:51,  3.98it/s]
24920it [1:26:51,  4.00it/s]
24921it [1:26:51,  3.99it/s]
24922it [1:26:51,  4.00it/s]
24923it [1:26:52,  3.97it/s]
24924it [1:26:52,  3.95it/s]
24925it [1:26:52,  3.96it/s]
24926it [1:26:52,  3.95it/s]
24927it [1:26:52,  4.70it/s]
24928it [1:26:53,  5.37it/s]
24929it [1:26:53,  6.07it/s]
24930it [1:26:53,  5.22it/s]
24931it [1:26:53,  4.71it/s]
24932it [1:26:53,  4.47it/s]
24933it [1:26:54,  4.32it/s]
24934it [1:26:54,  4.22it/s]
24935it [1:26:54,  4.15it/s]
24936it [1:26:55,  4.06it/s]
24937it [1:26:55,  4.03it/s]
24938it [1:26:55,  4.77it/s]
24939it [1:26:

25187it [1:27:41,  8.53it/s]
25188it [1:27:41,  8.61it/s]
25189it [1:27:41,  8.64it/s]
25190it [1:27:41,  8.59it/s]
25191it [1:27:41,  8.58it/s]
25192it [1:27:42,  8.57it/s]
25193it [1:27:42,  8.62it/s]
25194it [1:27:42,  8.65it/s]
25195it [1:27:42,  8.54it/s]
25196it [1:27:42,  8.59it/s]
25197it [1:27:42,  8.59it/s]
25198it [1:27:42,  8.65it/s]
25199it [1:27:42,  8.54it/s]
25200it [1:27:43,  8.45it/s]
25201it [1:27:43,  8.40it/s]
25202it [1:27:43,  8.38it/s]
25203it [1:27:43,  8.47it/s]
25204it [1:27:43,  8.41it/s]
25205it [1:27:43,  6.19it/s]
25206it [1:27:44,  5.32it/s]
25207it [1:27:44,  4.81it/s]
25208it [1:27:44,  4.50it/s]
25209it [1:27:44,  4.31it/s]
25210it [1:27:45,  4.21it/s]
25211it [1:27:45,  4.16it/s]
25212it [1:27:45,  4.08it/s]
25213it [1:27:45,  4.03it/s]
25214it [1:27:46,  4.00it/s]
25215it [1:27:46,  4.77it/s]
25216it [1:27:46,  5.53it/s]
25217it [1:27:46,  6.17it/s]
25218it [1:27:46,  6.74it/s]
25219it [1:27:46,  7.23it/s]
25220it [1:27:46,  7.60it/s]
25221it [1:27:

25469it [1:28:29,  8.37it/s]
25470it [1:28:30,  8.41it/s]
25471it [1:28:30,  8.44it/s]
25472it [1:28:30,  6.33it/s]
25473it [1:28:30,  5.32it/s]
25474it [1:28:30,  4.80it/s]
25475it [1:28:31,  4.37it/s]
25476it [1:28:31,  4.16it/s]
25477it [1:28:31,  4.11it/s]
25478it [1:28:32,  3.94it/s]
25479it [1:28:32,  3.93it/s]
25480it [1:28:32,  3.93it/s]
25481it [1:28:32,  3.94it/s]
25482it [1:28:33,  3.93it/s]
25483it [1:28:33,  3.94it/s]
25484it [1:28:33,  3.96it/s]
25485it [1:28:33,  3.93it/s]
25486it [1:28:34,  3.93it/s]
25487it [1:28:34,  3.95it/s]
25488it [1:28:34,  3.98it/s]
25489it [1:28:34,  3.96it/s]
25490it [1:28:35,  3.95it/s]
25491it [1:28:35,  3.95it/s]
25492it [1:28:35,  3.95it/s]
25493it [1:28:35,  3.96it/s]
25494it [1:28:36,  3.98it/s]
25495it [1:28:36,  4.75it/s]
25496it [1:28:36,  5.51it/s]
25497it [1:28:36,  6.16it/s]
25498it [1:28:36,  6.70it/s]
25499it [1:28:36,  5.58it/s]
25500it [1:28:37,  4.99it/s]
25501it [1:28:37,  5.70it/s]
25502it [1:28:37,  6.34it/s]
25503it [1:28:

In [25]:
tweets_finance_volume=feat_eng.create_volumesum_columns(twitter_data,'created_at_utc',sp500_data,'DATETIME','SIZE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3),('hours',6),('days',1),('days',5),('days',10),('days',30)],method='forward')


0it [00:00, ?it/s]
1it [00:00,  1.05it/s]
2it [00:01,  1.10it/s]
3it [00:02,  1.14it/s]
4it [00:03,  1.18it/s]
5it [00:04,  1.20it/s]
6it [00:04,  1.22it/s]
7it [00:05,  1.22it/s]
8it [00:06,  1.23it/s]
9it [00:07,  1.25it/s]
10it [00:08,  1.25it/s]
11it [00:08,  1.26it/s]
12it [00:09,  1.26it/s]
13it [00:10,  1.26it/s]
14it [00:11,  1.26it/s]
15it [00:12,  1.26it/s]
16it [00:12,  1.25it/s]
17it [00:13,  1.26it/s]
18it [00:14,  1.26it/s]
19it [00:15,  1.25it/s]
20it [00:16,  1.26it/s]
21it [00:16,  1.26it/s]
22it [00:17,  1.26it/s]
23it [00:18,  1.27it/s]
24it [00:19,  1.27it/s]
25it [00:19,  1.26it/s]
26it [00:20,  1.27it/s]
27it [00:21,  1.26it/s]
28it [00:22,  1.27it/s]
29it [00:23,  1.26it/s]
30it [00:23,  1.25it/s]
31it [00:24,  1.25it/s]
32it [00:25,  1.25it/s]
33it [00:26,  1.25it/s]
34it [00:27,  1.26it/s]
35it [00:27,  1.26it/s]
36it [00:28,  1.26it/s]
37it [00:29,  1.27it/s]
38it [00:30,  1.26it/s]
39it [00:31,  1.27it/s]
40it [00:31,  1.27it/s]
41it [00:32,  1.27it/s]
42it 

332it [04:36,  1.04s/it]
333it [04:37,  1.03s/it]
334it [04:38,  1.03s/it]
335it [04:39,  1.05s/it]
336it [04:40,  1.05s/it]
337it [04:41,  1.04s/it]
338it [04:42,  1.06s/it]
339it [04:43,  1.04s/it]
340it [04:44,  1.10s/it]
341it [04:45,  1.10s/it]
342it [04:46,  1.11s/it]
343it [04:48,  1.10s/it]
344it [04:49,  1.09s/it]
345it [04:50,  1.06s/it]
346it [04:51,  1.04s/it]
347it [04:52,  1.05s/it]
348it [04:53,  1.07s/it]
349it [04:54,  1.07s/it]
350it [04:55,  1.04s/it]
351it [04:56,  1.04s/it]
352it [04:57,  1.02s/it]
353it [04:58,  1.03it/s]
354it [04:59,  1.03it/s]
355it [05:00,  1.08it/s]
356it [05:00,  1.14it/s]
357it [05:01,  1.18it/s]
358it [05:02,  1.21it/s]
359it [05:03,  1.23it/s]
360it [05:03,  1.24it/s]
361it [05:04,  1.26it/s]
362it [05:05,  1.27it/s]
363it [05:06,  1.28it/s]
364it [05:06,  1.28it/s]
365it [05:07,  1.28it/s]
366it [05:08,  1.29it/s]
367it [05:09,  1.29it/s]
368it [05:10,  1.28it/s]
369it [05:10,  1.28it/s]
370it [05:11,  1.29it/s]
371it [05:12,  1.28it/s]


659it [09:00,  1.28it/s]
660it [09:01,  1.29it/s]
661it [09:01,  1.29it/s]
662it [09:02,  1.30it/s]
663it [09:03,  1.30it/s]
664it [09:04,  1.30it/s]
665it [09:04,  1.29it/s]
666it [09:05,  1.28it/s]
667it [09:06,  1.28it/s]
668it [09:07,  1.27it/s]
669it [09:08,  1.28it/s]
670it [09:08,  1.28it/s]
671it [09:09,  1.28it/s]
672it [09:10,  1.29it/s]
673it [09:11,  1.29it/s]
674it [09:11,  1.29it/s]
675it [09:12,  1.30it/s]
676it [09:13,  1.30it/s]
677it [09:14,  1.30it/s]
678it [09:15,  1.27it/s]
679it [09:15,  1.28it/s]
680it [09:16,  1.28it/s]
681it [09:17,  1.28it/s]
682it [09:18,  1.28it/s]
683it [09:18,  1.28it/s]
684it [09:19,  1.29it/s]
685it [09:20,  1.29it/s]
686it [09:21,  1.28it/s]
687it [09:22,  1.28it/s]
688it [09:22,  1.28it/s]
689it [09:23,  1.29it/s]
690it [09:24,  1.29it/s]
691it [09:25,  1.28it/s]
692it [09:25,  1.28it/s]
693it [09:26,  1.29it/s]
694it [09:27,  1.28it/s]
695it [09:28,  1.28it/s]
696it [09:29,  1.28it/s]
697it [09:29,  1.28it/s]
698it [09:30,  1.25it/s]


986it [13:18,  1.27it/s]
987it [13:19,  1.28it/s]
988it [13:19,  1.29it/s]
989it [13:20,  1.29it/s]
990it [13:21,  1.29it/s]
991it [13:22,  1.27it/s]
992it [13:23,  1.25it/s]
993it [13:23,  1.26it/s]
994it [13:24,  1.27it/s]
995it [13:25,  1.27it/s]
996it [13:26,  1.29it/s]
997it [13:27,  1.29it/s]
998it [13:27,  1.27it/s]
999it [13:28,  1.27it/s]
1000it [13:29,  1.28it/s]
1001it [13:30,  1.24it/s]
1002it [13:31,  1.22it/s]
1003it [13:31,  1.24it/s]
1004it [13:32,  1.26it/s]
1005it [13:33,  1.27it/s]
1006it [13:34,  1.27it/s]
1007it [13:35,  1.26it/s]
1008it [13:35,  1.26it/s]
1009it [13:36,  1.26it/s]
1010it [13:37,  1.27it/s]
1011it [13:38,  1.27it/s]
1012it [13:38,  1.27it/s]
1013it [13:39,  1.28it/s]
1014it [13:40,  1.28it/s]
1015it [13:41,  1.29it/s]
1016it [13:42,  1.29it/s]
1017it [13:42,  1.28it/s]
1018it [13:43,  1.29it/s]
1019it [13:44,  1.25it/s]
1020it [13:45,  1.24it/s]
1021it [13:46,  1.23it/s]
1022it [13:46,  1.25it/s]
1023it [13:47,  1.26it/s]
1024it [13:48,  1.26it/s]


1301it [17:27,  1.28it/s]
1302it [17:27,  1.26it/s]
1303it [17:28,  1.28it/s]
1304it [17:29,  1.27it/s]
1305it [17:30,  1.26it/s]
1306it [17:30,  1.27it/s]
1307it [17:31,  1.28it/s]
1308it [17:32,  1.28it/s]
1309it [17:33,  1.29it/s]
1310it [17:34,  1.28it/s]
1311it [17:34,  1.28it/s]
1312it [17:35,  1.29it/s]
1313it [17:36,  1.28it/s]
1314it [17:37,  1.28it/s]
1315it [17:37,  1.29it/s]
1316it [17:38,  1.28it/s]
1317it [17:39,  1.29it/s]
1318it [17:40,  1.29it/s]
1319it [17:41,  1.29it/s]
1320it [17:41,  1.29it/s]
1321it [17:42,  1.29it/s]
1322it [17:43,  1.29it/s]
1323it [17:44,  1.24it/s]
1324it [17:45,  1.25it/s]
1325it [17:45,  1.26it/s]
1326it [17:46,  1.27it/s]
1327it [17:47,  1.28it/s]
1328it [17:48,  1.27it/s]
1329it [17:48,  1.28it/s]
1330it [17:49,  1.28it/s]
1331it [17:50,  1.28it/s]
1332it [17:51,  1.28it/s]
1333it [17:52,  1.27it/s]
1334it [17:52,  1.27it/s]
1335it [17:53,  1.21it/s]
1336it [17:54,  1.24it/s]
1337it [17:55,  1.25it/s]
1338it [17:56,  1.28it/s]
1339it [17:5

...

24388it [5:16:15,  1.31it/s]
24389it [5:16:16,  1.32it/s]
24390it [5:16:17,  1.31it/s]
24391it [5:16:17,  1.32it/s]
24392it [5:16:18,  1.33it/s]
24393it [5:16:19,  1.33it/s]
24394it [5:16:20,  1.34it/s]
24395it [5:16:20,  1.35it/s]
24396it [5:16:21,  1.35it/s]
24397it [5:16:22,  1.35it/s]
24398it [5:16:23,  1.34it/s]
24399it [5:16:23,  1.34it/s]
24400it [5:16:24,  1.34it/s]
24401it [5:16:25,  1.35it/s]
24402it [5:16:26,  1.33it/s]
24403it [5:16:26,  1.34it/s]
24404it [5:16:27,  1.34it/s]
24405it [5:16:28,  1.35it/s]
24406it [5:16:29,  1.35it/s]
24407it [5:16:29,  1.35it/s]
24408it [5:16:30,  1.34it/s]
24409it [5:16:31,  1.32it/s]
24410it [5:16:32,  1.31it/s]
24411it [5:16:32,  1.31it/s]
24412it [5:16:33,  1.32it/s]
24413it [5:16:34,  1.33it/s]
24414it [5:16:35,  1.34it/s]
24415it [5:16:35,  1.33it/s]
24416it [5:16:36,  1.34it/s]
24417it [5:16:37,  1.34it/s]
24418it [5:16:38,  1.34it/s]
24419it [5:16:38,  1.35it/s]
24420it [5:16:39,  1.35it/s]
24421it [5:16:40,  1.35it/s]
24422it [5:16:

24670it [5:19:47,  1.33it/s]
24671it [5:19:48,  1.33it/s]
24672it [5:19:48,  1.34it/s]
24673it [5:19:49,  1.34it/s]
24674it [5:19:50,  1.34it/s]
24675it [5:19:51,  1.34it/s]
24676it [5:19:51,  1.34it/s]
24677it [5:19:52,  1.34it/s]
24678it [5:19:53,  1.34it/s]
24679it [5:19:54,  1.35it/s]
24680it [5:19:55,  1.17it/s]
24681it [5:19:56,  1.12it/s]
24682it [5:19:57,  1.10it/s]
24683it [5:19:58,  1.07it/s]
24684it [5:19:58,  1.10it/s]
24685it [5:19:59,  1.14it/s]
24686it [5:20:00,  1.19it/s]
24687it [5:20:01,  1.24it/s]
24688it [5:20:02,  1.25it/s]
24689it [5:20:02,  1.27it/s]
24690it [5:20:03,  1.29it/s]
24691it [5:20:04,  1.31it/s]
24692it [5:20:05,  1.27it/s]
24693it [5:20:05,  1.25it/s]
24694it [5:20:06,  1.28it/s]
24695it [5:20:07,  1.30it/s]
24696it [5:20:08,  1.31it/s]
24697it [5:20:08,  1.33it/s]
24698it [5:20:09,  1.33it/s]
24699it [5:20:10,  1.34it/s]
24700it [5:20:11,  1.33it/s]
24701it [5:20:11,  1.33it/s]
24702it [5:20:12,  1.33it/s]
24703it [5:20:13,  1.33it/s]
24704it [5:20:

24952it [5:23:23,  1.33it/s]
24953it [5:23:24,  1.33it/s]
24954it [5:23:24,  1.33it/s]
24955it [5:23:25,  1.34it/s]
24956it [5:23:26,  1.35it/s]
24957it [5:23:26,  1.35it/s]
24958it [5:23:27,  1.35it/s]
24959it [5:23:28,  1.35it/s]
24960it [5:23:29,  1.35it/s]
24961it [5:23:29,  1.36it/s]
24962it [5:23:30,  1.35it/s]
24963it [5:23:31,  1.35it/s]
24964it [5:23:32,  1.34it/s]
24965it [5:23:32,  1.34it/s]
24966it [5:23:33,  1.35it/s]
24967it [5:23:34,  1.36it/s]
24968it [5:23:35,  1.33it/s]
24969it [5:23:35,  1.33it/s]
24970it [5:23:36,  1.35it/s]
24971it [5:23:37,  1.35it/s]
24972it [5:23:38,  1.36it/s]
24973it [5:23:38,  1.35it/s]
24974it [5:23:39,  1.35it/s]
24975it [5:23:40,  1.35it/s]
24976it [5:23:41,  1.35it/s]
24977it [5:23:41,  1.35it/s]
24978it [5:23:42,  1.34it/s]
24979it [5:23:43,  1.33it/s]
24980it [5:23:44,  1.30it/s]
24981it [5:23:44,  1.31it/s]
24982it [5:23:45,  1.32it/s]
24983it [5:23:46,  1.33it/s]
24984it [5:23:47,  1.31it/s]
24985it [5:23:47,  1.31it/s]
24986it [5:23:

25234it [5:26:56,  1.34it/s]
25235it [5:26:57,  1.34it/s]
25236it [5:26:58,  1.35it/s]
25237it [5:26:59,  1.34it/s]
25238it [5:26:59,  1.34it/s]
25239it [5:27:00,  1.34it/s]
25240it [5:27:01,  1.34it/s]
25241it [5:27:02,  1.32it/s]
25242it [5:27:02,  1.33it/s]
25243it [5:27:03,  1.34it/s]
25244it [5:27:04,  1.34it/s]
25245it [5:27:05,  1.34it/s]
25246it [5:27:05,  1.34it/s]
25247it [5:27:06,  1.33it/s]
25248it [5:27:07,  1.34it/s]
25249it [5:27:08,  1.34it/s]
25250it [5:27:08,  1.34it/s]
25251it [5:27:09,  1.35it/s]
25252it [5:27:10,  1.34it/s]
25253it [5:27:11,  1.33it/s]
25254it [5:27:11,  1.33it/s]
25255it [5:27:12,  1.33it/s]
25256it [5:27:13,  1.33it/s]
25257it [5:27:14,  1.33it/s]
25258it [5:27:14,  1.33it/s]
25259it [5:27:15,  1.34it/s]
25260it [5:27:16,  1.35it/s]
25261it [5:27:17,  1.22it/s]
25262it [5:27:18,  1.23it/s]
25263it [5:27:19,  1.16it/s]
25264it [5:27:19,  1.20it/s]
25265it [5:27:20,  1.22it/s]
25266it [5:27:21,  1.24it/s]
25267it [5:27:22,  1.25it/s]
25268it [5:27:

25516it [5:30:30,  1.35it/s]
25517it [5:30:30,  1.34it/s]
25518it [5:30:31,  1.34it/s]
25519it [5:30:32,  1.33it/s]
25520it [5:30:33,  1.34it/s]
25521it [5:30:33,  1.34it/s]
25522it [5:30:34,  1.34it/s]
25523it [5:30:35,  1.34it/s]
25524it [5:30:36,  1.34it/s]
25525it [5:30:36,  1.34it/s]
25526it [5:30:37,  1.33it/s]
25527it [5:30:38,  1.32it/s]
25528it [5:30:39,  1.32it/s]
25529it [5:30:39,  1.32it/s]
25530it [5:30:40,  1.32it/s]
25531it [5:30:41,  1.31it/s]
25532it [5:30:42,  1.21it/s]
25533it [5:30:43,  1.16it/s]
25534it [5:30:44,  1.08it/s]
25535it [5:30:45,  1.05it/s]
25536it [5:30:46,  1.04it/s]
25537it [5:30:47,  1.05it/s]
25538it [5:30:48,  1.06it/s]
25539it [5:30:49,  1.05it/s]
25540it [5:30:50,  1.05it/s]
25541it [5:30:51,  1.08it/s]
25542it [5:30:51,  1.14it/s]
25543it [5:30:52,  1.19it/s]
25544it [5:30:53,  1.24it/s]
25545it [5:30:53,  1.28it/s]
25546it [5:30:54,  1.30it/s]
25547it [5:30:55,  1.31it/s]
25548it [5:30:56,  1.33it/s]
25549it [5:30:56,  1.33it/s]
25550it [5:30:

In [ ]:
tweets_finance.to_csv('tweets_finance.csv',index=True)

In [28]:
twitter_data_with_finance=twitter_data.join(tweets_finance)

In [29]:
twitter_data_with_finance=twitter_data_with_finance.join(tweets_finance_volume)

In [172]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance.copy()

In [173]:
twitter_data_with_finance_for_analysis.drop(['coordinates','extended_entities','geo','hashtags','in_reply_to_screen_name',
                                             'in_reply_to_status_id','in_reply_to_user_id','is_quote_status','is_retweet',
                                             'lang','place','possibly_sensitive','quoted_status','quoted_status_id','id',
                                             'retweeted_status','retrieved_utc'],1,inplace=True)

In [174]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(pd.get_dummies(twitter_data_with_finance_for_analysis['source']))

In [175]:
twitter_data_with_finance_for_analysis.drop(['source','truncated','user_mentions'],1,inplace=True)

In [176]:
fin_topic_words=['deal','deals','china','chinese','nafta','market','markets','stock','stocks','trade','trades','trading','tax',
             'taxes','taxation','rate','rates','unemployment','jobs','manafacture','manafacturers','dollar','dollars','$',
             'billion','billions','consumer','consumers','reserve','gdp','growth','bank','banks','debt','factories',
             'revenue','spending','deficit','economy','economies','economist','economic','economists','money',
             'tariff','tariffs','companies','inflation','refinance','finance','financial','currency','depreciating',
             'depreceate','fed','business','businesses','bankruptcy','price','prices','products','opec','investment',
             'savings','cents','cent','corporate','corporates','deficits','dow','nasdaq','labor','purchase','500',
             'industry','industries']

In [177]:
financial_words_matrix=feat_eng.token_matrix(twitter_data_with_finance_for_analysis['text'],fin_topic_words)

In [178]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(financial_words_matrix)

In [181]:
twitter_data_with_finance_for_analysis['trump_president']=twitter_data_with_finance_for_analysis['created_at_utc']>='2016-11-08 22:00:00'

In [146]:
twitter_data_with_finance_for_analysis.drop('text',1,inplace=True)

In [168]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(pd.get_dummies(twitter_data_with_finance_for_analysis['lda_topic']))

In [170]:
twitter_data_with_finance_for_analysis.drop('lda_topic',1,inplace=True)

In [171]:
twitter_data_with_finance_for_analysis.to_csv('twitter_data_with_finance_for_analysis.csv')